# ResNet18 on RMSPropBuiltin

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

from models.resnet18 import ResNet18
from optimizers.rmsprop_builtin import RMSPropBuiltin
from utils.training import train_model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Prepare CIFAR100 Data

https://github.com/abhinav-neil/resnet-transfer-learning/blob/main/code/cifar100_utils.py

In [ ]:
# Data preprocessing
transform_train = transforms.Compose(
    [
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]
)

transform_test = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)

# CIFAR-100 Dataset
train_dataset = torchvision.datasets.CIFAR100(root="./data", train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR100(root="./data", train=False, download=True, transform=transform_test)

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

## Training

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_class = RMSPropBuiltin

In [ ]:
num_epochs = 100
print_epochs = 10

In [ ]:
columns = ["Epoch", "Train_Loss", "Train_Accuracy", "Test_Accuracy", "Test_Loss"]
results_df = pd.DataFrame(columns=columns)

In [ ]:
model = ResNet18().to(device)

# Train the model
results = train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    device=device,
    criterion=criterion,
    optimizer_class=optimizer_class,
    optimizer_kwargs={},
    num_epochs=num_epochs,
    print_epochs=print_epochs,
)

# Append results to DataFrames
for epoch in range(num_epochs):
    results_df = pd.concat(
        [
            results_df,
            pd.DataFrame(
                [
                    {
                        "Epoch": epoch + 1,
                        "Train_Loss": results["train_losses"][epoch],
                        "Train_Accuracy": results["train_accuracies"][epoch],
                        "Test_Accuracy": results["test_accuracies"][epoch],
                        "Test_Loss": results["test_losses"][epoch],
                    }
                ]
            ),
        ]
    )


print("All experiments completed.")

In [ ]:
results_df.to_csv("results/rmsprop_builtin_resnet18_results.csv", index=False)